In [1]:
from __future__ import absolute_import, division, print_function
from __future__ import unicode_literals

In [2]:
import tensorflow as tf
import numpy as np
import copy
import tqdm
import IProgress
from hfunc import models
from hfunc import metrics

In [3]:
fashion_mnist = tf.keras.datasets.fashion_mnist
class_accuracy = metrics.ClassAccuracy()

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [5]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    optimizer='adam',
    loss='categorical_cross_entropy',
    metrics=['accuracy', tf.keras.metrics.AUC()]
)

In [7]:
tmp = model.get_weights()
print(tmp)

[array([[-0.04140123,  0.02092297,  0.07910059, ...,  0.01431214,
         0.07908142, -0.07038791],
       [ 0.02322625, -0.04434475, -0.04943981, ..., -0.00874022,
        -0.03492042, -0.07444277],
       [-0.05422504,  0.07055966, -0.04144818, ...,  0.03295391,
        -0.04739096, -0.05361735],
       ...,
       [ 0.01923709, -0.01245946,  0.0348358 , ..., -0.05117496,
        -0.05010323, -0.07360099],
       [ 0.05129444, -0.01525372, -0.05118312, ...,  0.04241318,
        -0.06969061,  0.07823717],
       [ 0.07548442,  0.03861769, -0.0103631 , ..., -0.00405139,
        -0.01304345,  0.0637982 ]], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [ ]:
l, a = model.evaluate(x_test, y_test, verbose=2, batch_size=256)
or_weights = model.get_weights()
tol_low = -1e-5
tol_high = 1e-5
num_zeros, num_worse, num_important = (0, 0, 0)
z = []
wr = []
imp = []
num_layers = 2
layer_sizes = [64, 128]
for layer_since_last, size in enumerate(layer_sizes):
    layer = num_layers - layer_since_last
    for i in range(size):
        w = copy.deepcopy(or_weights)
        w[0][:,i] = 0
        w[1][i] = 0
        w[2][i,:] = 0
        tester_model.set_weights(w)
        nl, na = tester_model2.evaluate(x_test, y_test, verbose=0, batch_size=256)
        print(f"Node {i}:", 1.*(na - a) + 0*(l - nl))
        change = l - nl
        if change <= tol_high and change >= tol_low:
            num_zeros += 1
            z += [i]
        elif change > 0:
            num_worse += 1
            wr += [i]
        else:
            num_important += 1
            imp += [i]

In [4]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10)
loss, acc = model.evaluate(x_test, y_test, verbose=2)
end_not_reached = True
improved = False
size = 128
tol = -1e-30
current_pos = 0
best_pos = -1
best_change = tol
original = model.get_weights()
bas = [acc]
bls = [loss]
best_weights = model.get_weights()
nodes_removed = []
best_acc = 0
best_loss = 1e20
ol = loss
oa = acc
num_removed = 0
while end_not_reached or improved:
    if not(end_not_reached):
        end_not_reached = True
        improved = False
        current_pos = 0
        size -= 1
        nodes_removed += [best_pos]
        best_weights[0][:,best_pos] = 0
        best_weights[1][best_pos] = 0
        best_weights[2][best_pos,:] = 0
        best_pos = -1
        tol -= best_change
        ol = best_loss
        oa = best_acc
        bas += [best_acc]
        bls += [best_loss]
        print("Improvement has occured!! Accuracy:", best_acc, "--- Loss:", best_loss, '--- Change:', best_change, '--- New tol:', tol)
        best_change = tol
        num_removed += 1
    if current_pos in nodes_removed:
        current_pos += 1
        if current_pos - num_removed >= size:
            end_not_reached = False
        continue
    w = copy.deepcopy(best_weights)
    w[0][:,current_pos] = 0
    w[1][current_pos] = 0
    w[2][current_pos,:] = 0
    model.set_weights(w)
    nl, na = model.evaluate(x_test, y_test, verbose=0)
    if 0.8*(na - oa) + 0.2*(ol - nl) >= best_change:
        best_change = 0.8*(na - oa) + 0.2*(ol - nl)
        print(best_change)
        best_pos = current_pos
        improved = True
        best_acc = na
        best_loss = nl
        print("Found something better")
    current_pos += 1
    if current_pos - num_removed >= size:
        end_not_reached = False
    if current_pos%20 == 0:
        print("Did 20 iterations")

model.set_weights(best_weights)
loss2, acc2 = model.evaluate(x_test, y_test, verbose=2)

iterations
0.0002429306507110596
Found something better
Did 20 iterations
Improvement has occured!! Accuracy: 0.8906000256538391 --- Loss: 0.32161515951156616 --- Change: 0.0002429306507110596 --- New tol: -0.005443316698074341
-0.0021391630172729495
Found something better
-0.0016697704792022706
Found something better
0.0
Found something better
7.74383544921875e-05
Found something better
Did 20 iterations
0.0001444399356842041
Found something better
Did 20 iterations
0.00019713044166564942
Found something better
Did 20 iterations
Did 20 iterations
Did 20 iterations
Did 20 iterations
Improvement has occured!! Accuracy: 0.8906999826431274 --- Loss: 0.32102933526039124 --- Change: 0.00019713044166564942 --- New tol: -0.00564044713973999
-0.002483254671096802
Found something better
-0.002012455463409424
Found something better
0.0
Found something better
6.430745124816895e-05
Found something better
Did 20 iterations
0.00015971064567565918
Found something better
Did 20 iterations
Did 20 itera

In [6]:
print(loss - loss2)
print(acc2 - acc)
print((loss - loss2)/loss * 100)
print((acc2 - acc)/acc * 100)
print(num_removed)
print(best_weights[1])

-0.005549728870391846
0.001399993896484375
-1.6980388999339908
0.15817352675649857
57
[ 4.5890132e-01 -1.3752976e-01  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00  4.9236059e-02  0.0000000e+00
  0.0000000e+00  6.4935058e-01  0.0000000e+00  0.0000000e+00
  2.7358076e-01  0.0000000e+00  0.0000000e+00  1.9085664e-01
  3.1870264e-01 -3.4913820e-01  1.6804899e-01  0.0000000e+00
 -4.4668129e-01  2.4767795e-01  2.1857737e-01  0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  2.9320963e-02
  2.7298591e-01 -3.1684133e-01  0.0000000e+00  1.1506892e-01
  4.7184643e-01  0.0000000e+00  0.0000000e+00  2.7141310e-04
  0.0000000e+00  2.2542518e-01  4.1248152e-01  6.2194014e-01
 -6.8283193e-02  0.0000000e+00  0.0000000e+00  0.0000000e+00
  6.0038650e-01  0.0000000e+00  2.6078582e-01  0.0000000e+00
  0.0000000e+00  0.0000000e+00  6.3469553e-01 -7.5865209e-02
  5.4575580e-01  2.0256771e-01  0.0000000e+00  2.5241551e-01
 -8.2020074e-02  4.8264244e-01  0.0000000e+00  6.1429638e-01

In [7]:
or_model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    
or_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
or_model.set_weights(original)
print(or_model.get_weights()[1])

[ 4.58901316e-01 -1.37529761e-01  5.09513974e-01 -1.15306377e-02
 -1.86158419e-01 -3.09453029e-02  4.92360592e-02  1.89392045e-01
  2.77105391e-01  6.49350584e-01 -1.61763560e-02  1.37479961e-01
  2.73580760e-01  2.62714744e-01 -1.65532231e-02  1.90856636e-01
  3.18702638e-01 -3.49138200e-01  1.68048993e-01 -1.09401187e-02
 -4.46681291e-01  2.47677952e-01  2.18577370e-01  3.25502306e-01
 -4.29187119e-01 -1.64396197e-01 -1.96925864e-01  2.93209627e-02
  2.72985905e-01 -3.16841334e-01 -6.65629432e-02  1.15068920e-01
  4.71846431e-01 -2.49029741e-01 -5.96166030e-03  2.71413097e-04
  2.90325373e-01  2.25425184e-01  4.12481517e-01  6.21940136e-01
 -6.82831928e-02  5.01024663e-01  2.75324076e-01  5.63114345e-01
  6.00386500e-01 -2.57192124e-02  2.60785818e-01 -3.08213145e-01
 -6.93467185e-02 -2.48844251e-01  6.34695530e-01 -7.58652091e-02
  5.45755804e-01  2.02567711e-01 -2.07985993e-02  2.52415508e-01
 -8.20200741e-02  4.82642442e-01 -1.99871942e-01  6.14296377e-01
 -1.71641279e-02  6.81132

In [8]:
model.evaluate(x_train, y_train, verbose=2)

1875/1875 - 3s - loss: 0.2440 - accuracy: 0.9126


[0.24402140080928802, 0.9126333594322205]

In [9]:
or_model.evaluate(x_train, y_train, verbose=2)

1875/1875 - 3s - loss: 0.2147 - accuracy: 0.9197


[0.21473637223243713, 0.9197333455085754]

In [34]:
model.evaluate(x_test, y_test, verbose=2)
y_pred = model.predict(x_test)
# class_accuracy.update_state(tf.convert_to_tensor(y_test), y_pred)
# print(class_accuracy.result())
K = len(set(y_test))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test == i] == y_test[y_test == i]).numpy())
    acc.append(a)
print(acc)

m = tf.keras.metrics.AUC()
m.update_state(tf.one_hot(y_test, 10), y_pred)
print(m.result())

313/313 - 1s - loss: 0.3324 - accuracy: 0.8865
[0.83, 0.973, 0.773, 0.885, 0.848, 0.959, 0.701, 0.966, 0.976, 0.954]
tf.Tensor(0.9924697, shape=(), dtype=float32)


In [35]:
or_model.evaluate(x_test, y_test, verbose=2)
y_pred = or_model.predict(x_test)
# class_accuracy.update_state(y_test, y_pred)
# print(class_accuracy.result())

K = len(set(y_test))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test == i] == y_test[y_test == i]).numpy())
    acc.append(a)
print(acc)

m = tf.keras.metrics.AUC()
m.update_state(tf.one_hot(y_test, 10), y_pred)
print(m.result())

313/313 - 1s - loss: 0.3268 - accuracy: 0.8851
[0.828, 0.967, 0.78, 0.901, 0.861, 0.961, 0.649, 0.97, 0.972, 0.962]
tf.Tensor(0.9917027, shape=(), dtype=float32)


In [44]:
red_model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128-num_removed, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    
red_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
red_model.fit(x_train, y_train, epochs=20, verbose=1)
red_model.evaluate(x_test, y_test, verbose=2)

y_pred = red_model.predict(x_test)

K = len(set(y_test))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test == i] == y_test[y_test == i]).numpy())
    acc.append(a)
print(acc)

m = tf.keras.metrics.AUC()
m.update_state(tf.one_hot(y_test, 10), y_pred)
print(m.result())

Epoch 1/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.5084 - accuracy: 0.8229
Epoch 2/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3844 - accuracy: 0.8634
Epoch 3/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3455 - accuracy: 0.8763
Epoch 4/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3237 - accuracy: 0.8824
Epoch 5/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3086 - accuracy: 0.8868
Epoch 6/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2934 - accuracy: 0.8930
Epoch 7/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2791 - accuracy: 0.8978
Epoch 8/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2700 - accuracy: 0.8999
Epoch 9/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2626 - accuracy: 0.9040
Epoch 10/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.255

In [37]:
import os
from tensorflow import keras
import datetime

In [38]:
reduced_path = f'C:/Users/lucas/Documents/Masters/models/reduced/fashion_mnist_128_{128-num_removed}'
original_path = f'C:/Users/lucas/Documents/Masters/models/orininal/fashion_mnist_128_{128-num_removed}'
full_reduced_path = f'C:/Users/lucas/Documents/Masters/models/full_reduced/fashion_mnist_128_{128-num_removed}'

In [39]:
model.save_weights(reduced_path)

In [40]:
or_model.save_weights(original_path)

In [41]:
new_weights = [np.zeros((best_weights[0].shape[0], best_weights[0].shape[1] - num_removed)), np.zeros((best_weights[1].shape[0] - num_removed)), np.zeros((best_weights[2].shape[0] - num_removed, best_weights[2].shape[1])), best_weights[3]]

j = 0
for i in range(len(best_weights[1])):
    if i not in nodes_removed:
        new_weights[0][:, j] = best_weights[0][:, i]
        new_weights[1][j] = best_weights[1][i]
        new_weights[2][j, :] = best_weights[2][i, :]
        j = j + 1

In [42]:
red_model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128-num_removed, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    
red_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
red_model.set_weights(new_weights)
red_model.save_weights(full_reduced_path)
red_model.evaluate(x_test, y_test, verbose=2)

313/313 - 1s - loss: 0.3324 - accuracy: 0.8865


[0.33238136768341064, 0.8865000009536743]

In [43]:
red_model.set_weights(new_weights)
red_model.fit(x_train, y_train, epochs=15, verbose=1, batch_size=4096)
red_model.evaluate(x_test, y_test, verbose=2, batch_size=256)

Epoch 1/15
15/15 [==============================] - 0s 23ms/step - loss: 0.2254 - accuracy: 0.9166
Epoch 2/15
15/15 [==============================] - 0s 21ms/step - loss: 0.2146 - accuracy: 0.9207
Epoch 3/15
15/15 [==============================] - 0s 24ms/step - loss: 0.2105 - accuracy: 0.9227
Epoch 4/15
15/15 [==============================] - 0s 18ms/step - loss: 0.2080 - accuracy: 0.9239
Epoch 5/15
15/15 [==============================] - 0s 17ms/step - loss: 0.2056 - accuracy: 0.9241
Epoch 6/15
15/15 [==============================] - 0s 22ms/step - loss: 0.2030 - accuracy: 0.9256
Epoch 7/15
15/15 [==============================] - 0s 17ms/step - loss: 0.2016 - accuracy: 0.9259
Epoch 8/15
15/15 [==============================] - 0s 17ms/step - loss: 0.1999 - accuracy: 0.9265
Epoch 9/15
15/15 [==============================] - 0s 17ms/step - loss: 0.1985 - accuracy: 0.9270
Epoch 10/15
15/15 [==============================] - 0s 20ms/step - loss: 0.1968 - accuracy: 0.9277
Epoch 11/

[0.31121399998664856, 0.8920000195503235]

In [45]:
new_model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
new_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
new_model.set_weights(best_weights)
new_model.evaluate(x_test, y_test, verbose=2)
new_model.fit(x_train, y_train, epochs=10, verbose=1, batch_size=2048)
new_model.evaluate(x_test, y_test, verbose=2)

313/313 - 1s - loss: 0.3324 - accuracy: 0.8865
Epoch 1/10
30/30 [==============================] - 1s 17ms/step - loss: 0.2230 - accuracy: 0.9175
Epoch 2/10
30/30 [==============================] - 0s 15ms/step - loss: 0.2123 - accuracy: 0.9214
Epoch 3/10
30/30 [==============================] - 0s 15ms/step - loss: 0.2089 - accuracy: 0.9231
Epoch 4/10
30/30 [==============================] - 0s 14ms/step - loss: 0.2059 - accuracy: 0.9247
Epoch 5/10
30/30 [==============================] - 0s 15ms/step - loss: 0.2034 - accuracy: 0.9251
Epoch 6/10
30/30 [==============================] - 0s 15ms/step - loss: 0.2008 - accuracy: 0.9259
Epoch 7/10
30/30 [==============================] - 0s 15ms/step - loss: 0.1989 - accuracy: 0.9267
Epoch 8/10
30/30 [==============================] - 0s 14ms/step - loss: 0.1976 - accuracy: 0.9272
Epoch 9/10
30/30 [==============================] - 0s 14ms/step - loss: 0.1952 - accuracy: 0.9277
Epoch 10/10
30/30 [==============================] - 0s 14ms/s

[0.31389063596725464, 0.8902999758720398]

In [46]:
loss, acc = new_model.evaluate(x_test, y_test, verbose=2)
end_not_reached = True
improved = False
size = 128
tol = -1e-30
current_pos = 0
best_pos = -1
best_change = tol
original2 = new_model.get_weights()
bas2 = [acc]
bls2 = [loss]
best_weights2 = new_model.get_weights()
nodes_removed2 = []
best_acc = 0
best_loss = 1e20
ol = loss
oa = acc
num_removed2 = 0
while end_not_reached or improved:
    if not(end_not_reached):
        end_not_reached = True
        improved = False
        current_pos = 0
        size -= 1
        nodes_removed2 += [best_pos]
        best_weights2[0][:,best_pos] = 0
        best_weights2[1][best_pos] = 0
        best_weights2[2][best_pos,:] = 0
        best_pos = -1
        tol -= best_change
        ol = best_loss
        oa = best_acc
        bas2 += [best_acc]
        bls2 += [best_loss]
        print("Improvement has occured!! Accuracy:", best_acc, "--- Loss:", best_loss, '--- Change:', best_change, '--- New tol:', tol)
        best_change = tol
        num_removed2 += 1
    if current_pos in nodes_removed2:
        current_pos += 1
        if current_pos - num_removed2 >= size:
            end_not_reached = False
        continue
    w = copy.deepcopy(best_weights2)
    w[0][:,current_pos] = 0
    w[1][current_pos] = 0
    w[2][current_pos,:] = 0
    new_model.set_weights(w)
    nl, na = new_model.evaluate(x_test, y_test, verbose=0)
    print(f"Node {current_pos}:", 0.8*(na - oa) + 0.2*(ol - nl))
    if 0.8*(na - oa) + 0.2*(ol - nl) > best_change:
        best_change = 0.8*(na - oa) + 0.2*(ol - nl)
        print(best_change)
        best_pos = current_pos
        improved = True
        best_acc = na
        best_loss = nl
        print("Found something better")
    current_pos += 1
    if current_pos - num_removed2 >= size:
        end_not_reached = False
    if current_pos%20 == 0:
        print("Did 20 iterations")

new_model.set_weights(best_weights2)
loss2, acc2 = new_model.evaluate(x_test, y_test, verbose=2)

313/313 - 1s - loss: 0.3139 - accuracy: 0.8903
Node 0: -0.0014394640922546387
Node 1: -0.002711939811706543
Node 2: 0.0
0.0
Found something better
Node 3: 0.0
Node 4: 0.0
Node 5: 0.0
Node 6: -0.0002560496330261231
Node 7: 0.0
Node 8: 0.0
Node 9: -0.0047052264213562015
Node 10: 0.0
Node 11: 0.0
Node 12: -0.00038411021232604987
Node 13: 0.0
Node 14: 0.0
Node 15: -0.0027272582054138185
Node 16: -0.002950209379196167
Node 17: -0.0006514310836791993
Node 18: -0.0029980480670928956
Node 19: 0.0
Did 20 iterations
Node 20: -0.0025429069995880128
Node 21: -0.0010326504707336426
Node 22: -0.00021579861640930176
Node 23: 0.0
Node 24: 0.0
Node 25: 0.0
Node 26: 0.0
Node 27: -0.005143517255783082
Node 28: -0.0032790184020996095
Node 29: -0.013628959655761719
Node 30: 0.0
Node 31: 0.0004489600658416748
0.0004489600658416748
Found something better
Node 32: -0.0017570316791534425
Node 33: 0.0
Node 34: 0.0
Node 35: 9.98377799987793e-06
Node 36: 0.0
Node 37: -0.0014190077781677248
Node 38: -0.00554142594

KeyboardInterrupt: 

In [56]:
tester_model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
tester_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [60]:
import time

In [61]:
l, a = or_model.evaluate(x_test, y_test, verbose=2)
or_weights = or_model.get_weights()
size = 128
start = time.time()
for i in range(128):
    w = copy.deepcopy(or_weights)
    w[0][:,i] = 0
    w[1][i] = 0
    w[2][i,:] = 0
    tester_model.set_weights(w)
    nl, na = tester_model.evaluate(x_test, y_test, verbose=0)
    print(f"Node {i}:", 0.6*(na - a) + 0.4*(l - nl))
end = time.time()
print(end - start)

313/313 - 1s - loss: 0.3268 - accuracy: 0.8851
Node 0: -0.0010176539421081544
Node 1: -0.0018407821655273436
Node 2: 0.00044121742248535154
Node 3: 0.0
Node 4: 7.524490356445313e-05
Node 5: -7.578134536743163e-05
Node 6: 0.0008309841156005859
Node 7: -0.00022814273834228515
Node 8: -0.0009253263473510743
Node 9: -0.003994774818420411
Node 10: 0.0
Node 11: 0.0007683992385864258
Node 12: -0.0011681914329528809
Node 13: 4.014968872070312e-05
Node 14: 0.0
Node 15: -0.0018353462219238281
Node 16: -0.0006745338439941406
Node 17: -0.001815140247344971
Node 18: -0.002695620059967041
Node 19: 0.0
Node 20: 0.0013562440872192383
Node 21: -0.0014670729637145995
Node 22: -0.0005298018455505371
Node 23: -0.001352214813232422
Node 24: 0.0001497507095336914
Node 25: -0.00017908811569213867
Node 26: -0.00031138658523559575
Node 27: -0.007463109493255616
Node 28: -0.0038578867912292484
Node 29: -0.002183234691619873
Node 30: -4.6706199645996095e-05
Node 31: -0.00023217201232910158
Node 32: -0.0005174160

In [62]:
l, a = or_model.evaluate(x_test, y_test, verbose=2)
size = 128
start = time.time()
for i in range(128):
    w = or_model.get_weights()
    w[0][:,i] = 0
    w[1][i] = 0
    w[2][i,:] = 0
    tester_model.set_weights(w)
    nl, na = tester_model.evaluate(x_test, y_test, verbose=0)
    print(f"Node {i}:", 0.6*(na - a) + 0.4*(l - nl))
end = time.time()
print(end - start)

313/313 - 1s - loss: 0.3268 - accuracy: 0.8851
Node 0: -0.0010176539421081544
Node 1: -0.0018407821655273436
Node 2: 0.00044121742248535154
Node 3: 0.0
Node 4: 7.524490356445313e-05
Node 5: -7.578134536743163e-05
Node 6: 0.0008309841156005859
Node 7: -0.00022814273834228515
Node 8: -0.0009253263473510743
Node 9: -0.003994774818420411
Node 10: 0.0
Node 11: 0.0007683992385864258
Node 12: -0.0011681914329528809
Node 13: 4.014968872070312e-05
Node 14: 0.0
Node 15: -0.0018353462219238281
Node 16: -0.0006745338439941406
Node 17: -0.001815140247344971
Node 18: -0.002695620059967041
Node 19: 0.0
Node 20: 0.0013562440872192383
Node 21: -0.0014670729637145995
Node 22: -0.0005298018455505371
Node 23: -0.001352214813232422
Node 24: 0.0001497507095336914
Node 25: -0.00017908811569213867
Node 26: -0.00031138658523559575
Node 27: -0.007463109493255616
Node 28: -0.0038578867912292484
Node 29: -0.002183234691619873
Node 30: -4.6706199645996095e-05
Node 31: -0.00023217201232910158
Node 32: -0.0005174160

In [ ]:
import tqdm

In [66]:
num_test = 20
num_zeros = np.zeros(num_test)
num_worse = np.zeros(num_test)
num_important = np.zeros(num_test)
losses = np.zeros(num_test)
accs = np.zeros(num_test)
zero_nodes = []
worsening_nodes = []
important_nodes = []
tol = -1e-4
for j in tqdm.trange(num_test):
    blank_model = tf.keras.models.Sequential([
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(10, activation='softmax')
        ])
    blank_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    blank_model.fit(x_train, y_train, epochs=10, verbose=0)
    l, a = blank_model.evaluate(x_test, y_test, verbose=0)
    losses[j] = l
    accs[j] = a
    z = []
    wr = []
    imp = []
    for i in range(128):
        w = blank_model.get_weights()
        w[0][:,i] = 0
        w[1][i] = 0
        w[2][i,:] = 0
        tester_model.set_weights(w)
        nl, na = tester_model.evaluate(x_test, y_test, verbose=0)
        change = 0.8*(na - a) + 0.2*(l - nl)
        if change <= 0 and change >= tol:
            num_zeros[j] += 1
            z += [i]
        elif change > 0:
            num_worse[j] += 1
            wr += [i]
        else:
            num_important[j] += 1
            imp += [i]
    zero_nodes += [z]
    worsening_nodes += [wr]
    important_nodes += [imp]

25%|██▌       | 5/20 [06:07<18:23, 73.57s/it]


KeyboardInterrupt: 

In [67]:
print(num_zeros)
print(num_worse)
print(num_important)

[25. 19. 20. 24. 20.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.]
[29. 29. 31. 21. 30.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.]
[74. 80. 77. 83. 78.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.]
